# **Chapter 15: Agent Frameworks in Practice – CrewAI vs. LangGraph**

### **From Theory to Implementation**
In the previous chapters, we established the **GenAI Maturity Model** and explored architectural patterns like **Fractal Chain of Thought (FCoT)**. Now, we move from *design* to *construction*.

While it is possible to build agents from scratch using raw Python and API calls (as we did in Chapter 13), modern **Agent Frameworks** provide powerful abstractions that handle state management, context, and tool orchestration. But which one should you choose?

### **The Use Case: Compliance-Aware Loan Origination**
To fairly compare these frameworks, we will implement the exact same business logic in both: **A Multi-Agent Loan Origination Pipeline**.

The workflow consists of four distinct stages:
1.  **Document Validator:** Ensures the JSON application has all required fields.
2.  **Credit Analyst:** Queries a (mock) bureau API for credit scores.
3.  **Risk Assessor:** Synthesizes income, debt, and credit data to assign a risk score.
4.  **Compliance Officer:** Checks the final risk score against regulatory policies to approve or deny the loan.

### **The Battle of Architectures**

In this notebook, you will build this pipeline twice to understand two fundamentally different design philosophies:

#### **1. CrewAI: The "Role-Playing" Approach**
* **Philosophy:** Agents are treated as "employees" with personas, backstories, and goals.
* **Architecture:** Hierarchical or Sequential. You assemble a "Crew" and let them collaborate naturally.
* **Best For:** Projects where you want high-level abstraction and "social" delegation between agents without managing every step of the control flow.

#### **2. LangGraph: The "State Machine" Approach**
* **Philosophy:** Agents are nodes in a graph, and execution is a flow of state.
* **Architecture:** Cyclic Graph (State Machine). You explicitly define `Nodes` (actions) and `Edges` (logic) to control exactly how data moves.
* **Best For:** Complex, deterministic workflows requiring loops, conditional branching (e.g., "If validation fails, go back to start"), and fine-grained control.

**This notebook implements a "Safe Mode" Execution Engine featuring:**
1.  **Active Throttling (`ratelimit`):** We strictly cap execution to ~15 calls/minute to stay within the Gemini Free Tier limits.
2.  **Exponential Backoff (`tenacity`):** If an API call fails, the system waits and retries with increasing delays, preventing crash loops.
3.  **Graceful Failure Handling:** Instead of dumping a Python stack trace, the agents produce structured "Mission Reports" explaining exactly why a process failed.


<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Agentic-Architectural-Patterns-for-Building-Multi-Agent-Systems/blob/main/Chapter_15/Chapter_15_Agents.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



In [ ]:
#@title Install dependencies

!pip install --upgrade crewai crewai_tools langchain-google-genai langgraph langchain-core tenacity ratelimit -q

In [5]:
#@title Imports and LLM Configuration

import os
import json
from getpass import getpass

from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
#@title API Setup

from getpass import getpass



GEMINI=getpass("Enter your GEMINI API KEY: ")
os.environ["GOOGLE_API_KEY"]=GEMINI
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

model= "gemini-2.5-flash"

# --- Initialize the LLM ---
try:
    llm = LLM(
        model=model,
        api_key=os.getenv("GOOGLE_API_KEY"),
        temperature=0.0
    )
    print(f"LLM ({llm.model}) configured.")
except Exception as e:
    print(f"Error initializing CrewAI LLM: {e}")
    llm = None

try:
    lg_llm = ChatGoogleGenerativeAI(model=model)
    print("LangGraph-specific LLM (lg_llm) initialized.")
except Exception as e:
    print(f"Error initializing LangGraph LLM: {e}")
    lg_llm = None

In [7]:
#@title Execution Engine
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception
from ratelimit import limits, sleep_and_retry
import time

# --- CONFIGURATION ---
CALLS = 15  # Max calls...
PERIOD = 60 # ...per minute

# --- HELPER: ERROR FILTER ---
def is_rate_limit_error(e):
    msg = str(e).lower()
    return "429" in msg or "quota" in msg or "resource exhausted" in msg or "serviceunavailable" in msg

# --- ROBUST WRAPPER ---
@sleep_and_retry
@limits(calls=CALLS, period=PERIOD)
@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=2, min=4, max=30),
    retry=retry_if_exception(is_rate_limit_error),
    reraise=True
)
def robust_execute(func, *args, **kwargs):
    """
    Executes any function (CrewAI kickoff, LangGraph invoke) with built-in
    rate limiting and auto-retries for transient API errors.
    """
    print(f"  >> [Clock {time.strftime('%X')}] Executing Agent Action (Safe Mode)...")
    return func(*args, **kwargs)

# --- ERROR HANDLER ---
def handle_execution_error(e):
    """Prints a clean, professional error report."""
    error_msg = str(e)
    is_quota = "429" in error_msg or "quota" in error_msg.lower()

    print("\n" + "━" * 60)
    print("  🛑  MISSION ABORTED: SYSTEM CRITICAL ERROR")
    print("━" * 60)

    if is_quota:
        print("  ⚠️   CAUSE:    QUOTA EXCEEDED (API Refusal)")
        print("  🔍   CONTEXT:  The LLM provider rejected the request.")
        print("\n  🛠️   ACTION:    [1] Wait before retrying")
        print("                  [2] Check API Limits (Free Tier is ~15 RPM)")
    else:
        print(f"  ⚠️   CAUSE:    UNEXPECTED EXCEPTION")
        print(f"  📝   DETAILS:  {error_msg}")

    print("━" * 60 + "\n")

## Common Tools Definition

First, we define the common set of Python functions that our agents will use as tools. Both frameworks will use these same tools.

In [21]:
#@title Common Tools & Mock Data
import json
from crewai.tools import BaseTool

# --- 1. HELPER: Mock Document Fetcher ---
def get_document_content(document_id: str) -> str:
    print(f"--- HELPER: Simulating fetch for doc_id: {document_id} ---")

    if document_id == "document_valid_123":
        # Happy Path: High Income, Good History
        return json.dumps({
            "customer_id": "CUST-12345",
            "loan_amount": 50000,
            "income": "USD 120000 a year",
            "credit_history": "7 years good standing"
        })

    elif document_id == "document_risky_789":
        # Unhappy Path: Valid Docs, but LOW CREDIT SCORE
        return json.dumps({
            "customer_id": "CUST-99999",
            "loan_amount": 50000,
            "income": "USD 40000 a year",
            "credit_history": "Recent Missed Payments"
        })

    elif document_id == "document_invalid_456":
        # Broken Path: Missing fields (income)
        return json.dumps({
            "customer_id": "CUST-55555",
            "loan_amount": 200000,
            "credit_history": "1 year"
        })
    else:
        return json.dumps({"error": "Document ID not found."})

# --- 2. TOOLS: With Logic for Risk/Compliance ---

class ValidateDocumentFieldsTool(BaseTool):
    name: str = "Validate Document Fields"
    description: str = "Validates JSON application data."
    def _run(self, application_data: str) -> str:
        print(f"--- TOOL: Validating document fields ---")
        try:
            data = json.loads(application_data)
            required = ["customer_id", "loan_amount", "income", "credit_history"]
            missing = [f for f in required if f not in data]
            if missing:
                return json.dumps({"error": f"Missing fields: {', '.join(missing)}"})
            return json.dumps({"status": "validated", "data": data})
        except:
            return json.dumps({"error": "Invalid JSON"})

class QueryCreditBureauAPITool(BaseTool):
    name: str = "Query Credit Bureau API"
    description: str = "Gets credit score for customer_id."
    def _run(self, customer_id: str) -> str:
        print(f"--- TOOL: Calling Credit Bureau for {customer_id} ---")
        scores = {
            "CUST-12345": 810, # Good
            "CUST-99999": 550, # BAD SCORE (< 600)
            "CUST-55555": 620
        }
        score = scores.get(customer_id)
        if score:
            return json.dumps({"customer_id": customer_id, "credit_score": score})
        return json.dumps({"error": "Customer not found"})

class CalculateRiskScoreTool(BaseTool):
    name: str = "Calculate Risk Score"
    description: str = "Calculates risk based on financial data."
    def _run(self, loan_amount: int, income: str, credit_score: int) -> str:
        print(f"--- TOOL: Calculating Risk (Score: {credit_score}) ---")
        # Logic: Credit Score < 600 is automatic HIGH risk
        if credit_score < 600:
            return json.dumps({"risk_score": 9, "reason": "Credit score too low"})

        # Standard logic
        try:
            inc_val = int(''.join(filter(str.isdigit, income)))
            ann_inc = inc_val * 12 if "month" in income.lower() else inc_val
        except: ann_inc = 0

        risk = 1
        if credit_score < 720: risk += 2
        if ann_inc > 0 and (loan_amount / ann_inc) > 0.5: risk += 3

        return json.dumps({"risk_score": min(risk, 10)})

class CheckLendingComplianceTool(BaseTool):
    name: str = "Check Lending Compliance"
    description: str = "Checks policy compliance."
    def _run(self, credit_history: str, risk_score: int) -> str:
        print(f"--- TOOL: Compliance Check (Risk: {risk_score}) ---")
        if risk_score >= 8:
            return json.dumps({"is_compliant": False, "reason": f"Risk Score {risk_score} exceeds limit of 7."})
        return json.dumps({"is_compliant": True, "reason": "Compliant."})

# Instantiate
validate_document_fields_tool = ValidateDocumentFieldsTool()
query_credit_bureau_api_tool = QueryCreditBureauAPITool()
calculate_risk_score_tool = CalculateRiskScoreTool()
check_lending_compliance_tool = CheckLendingComplianceTool()

In [22]:
#@title Helper Functions

def get_document_content(document_id: str) -> str:
    """
    Simulates fetching document content based on its ID.
    """
    print(f"--- HELPER: Simulating fetch for doc_id: {document_id} ---")
    if document_id == "document_valid_123":
        data = {
            "customer_id": "CUST-12345",
            "loan_amount": 50000,
            "income": "USD 120000 a year",
            "credit_history": "7 years"
        }
        return json.dumps(data)
    elif document_id == "document_invalid_456":
        data = {
            "customer_id": "CUST-55555",
            "loan_amount": 200000,
            # "income" is missing
            "credit_history": "1 year"
        }
        return json.dumps(data)
    else:
        return json.dumps({"error": "Document ID not found."})

# Implementation 1: CrewAI (The Collaborative Team)

CrewAI's philosophy is centered on **role-playing agents** that form a "crew."

In [11]:
#@title Define CrewAI Agents

# 1. Document Validation Agent
doc_specialist = Agent(
    role="Document Validation Specialist",
    goal="Validate the completeness and format of a new loan application provided as a JSON string.",
    backstory=("You are a meticulous agent responsible for the first step of loan processing."),
    tools=[validate_document_fields_tool],
    llm=llm,
    verbose=True
)

# 2. Credit Check Agent
credit_analyst = Agent(
    role="Credit Check Agent",
    goal="Query the credit bureau API to retrieve an applicant's credit score.",
    backstory=("You are a specialized agent that interacts with the Credit Bureau."),
    tools=[query_credit_bureau_api_tool],
    llm=llm,
    verbose=True
)

# 3. Risk Assessment Agent
risk_assessor = Agent(
    role="Risk Assessment Analyst",
    goal="Calculate the financial risk score for a loan application.",
    backstory=("You are a quantitative analyst agent."),
    tools=[calculate_risk_score_tool],
    llm=llm,
    verbose=True
)

# 4. Compliance Agent
compliance_officer = Agent(
    role="Compliance Officer",
    goal="Check the application against all internal lending policies and compliance rules.",
    backstory=("You are the final checkpoint for policy and compliance."),
    tools=[check_lending_compliance_tool],
    llm=llm,
    verbose=True
)

# 5. Manager Agent
manager = Agent(
    role="Loan Processing Manager",
    goal="Manage the loan application workflow and compile the final report.",
    backstory=("You are the manager responsible for orchestrating the loan processing pipeline."),
    llm=llm,
    allow_delegation=True,
    verbose=True
)

print("Agents defined successfully!")

Agents defined successfully!


In [14]:
#@title Define CrewAI Tasks

loan_application_inputs_valid = {
    "applicant_id": "borrower_good_780",
    "document_id": "document_valid_123"
}

loan_application_inputs_invalid = {
    "applicant_id": "borrower_bad_620",
    "document_id": "document_invalid_456"
}

if llm:
    task_validate = Task(
        description=(
            "Validate the loan application provided as a JSON string: '{document_content}'. "
            "Pass this string to the 'Validate Document Fields' tool."
        ),
        expected_output="A JSON string with the validation status.",
        agent=doc_specialist
    )

    task_credit = Task(
        description="Extract customer_id and call Query Credit Bureau API.",
        expected_output="A JSON string containing the credit_score.",
        agent=credit_analyst,
        context=[task_validate]
    )

    task_risk = Task(
        description="Extract loan details and credit score, then Calculate Risk Score.",
        expected_output="A JSON string containing the risk_score.",
        agent=risk_assessor,
        context=[task_validate, task_credit]
    )

    task_compliance = Task(
        description="Check Lending Compliance based on history and risk score.",
        expected_output="Compliance status JSON.",
        agent=compliance_officer,
        context=[task_validate, task_risk]
    )

    task_report = Task(
        description="Compile a final report with Approve/Deny decision.",
        expected_output="Markdown report.",
        agent=manager,
        allow_delegation=False,
        context=[task_validate, task_credit, task_risk, task_compliance]
    )
    print("Tasks defined.")

Tasks defined.


In [ ]:
#@title Run CrewAI

if llm is None:
    print("LLM not initialized.")
else:
    loan_crew = Crew(
        agents=[doc_specialist, credit_analyst, risk_assessor, compliance_officer],
        tasks=[task_validate, task_credit, task_risk, task_compliance, task_report],
        process=Process.hierarchical,
        manager_agent=manager,
        verbose=True
    )

    try:
        print("--- KICKING OFF CREWAI (VALID INPUTS) ---")
        valid_json = get_document_content(loan_application_inputs_valid['document_id'])
        inputs = {'document_content': valid_json}

        # WRAPPING THE KICKOFF IN ROBUST EXECUTION
        result = robust_execute(loan_crew.kickoff, inputs=inputs)

        print("\n\n--- FINAL REPORT ---")
        print(result)

    except Exception as e:
        handle_execution_error(e)

In [ ]:
#@title Run CrewAI (Unhappy Path: Low Credit Score)
if llm:
    print("--- KICKING OFF CREWAI (LOW CREDIT SCENARIO) ---")

    # 1. Fetch the "Risky" document (Valid JSON, but Borrower has bad credit)
    # Ensure you updated the 'Common Tools' cell to include 'document_risky_789' first!
    risky_json = get_document_content("document_risky_789")

    inputs_risky = {
        'document_content': risky_json
    }

    try:
        # 2. Execute with Robustness Wrapper
        result = robust_execute(loan_crew.kickoff, inputs=inputs_risky)

        print("\n\n--- FINAL REPORT (SHOULD DENY) ---")
        print(result)

    except Exception as e:
        handle_execution_error(e)

# Implementation 2: LangGraph

LangGraph's philosophy is to define a **stateful graph**.

In [17]:
#@title 2.2: Define LangGraph Nodes

import typing
import json
from langchain_core.messages import HumanMessage

# 1. Define the Graph State
class LoanGraphState(typing.TypedDict):
    applicant_id: str
    document_id: str
    document_content: str
    validation_status: str
    customer_id: str
    loan_amount: int
    income: str
    credit_history: str
    credit_score: int
    risk_score: int
    risk_level: str
    compliance_status: str
    final_decision: str
    error: str

# Node 0: Fetch Document
def node_fetch_document(state: LoanGraphState):
    print("--- NODE: Fetching Document ---")
    try:
        content = get_document_content(state["document_id"])
        return {"document_content": content}
    except Exception as e:
        return {"error": str(e)}

# Node 1: Validate Document
def node_validate_document(state: LoanGraphState):
    print("--- NODE: Validating Document ---")
    try:
        res = json.loads(validate_document_fields_tool._run(state["document_content"]))
        if "error" in res:
            return {"validation_status": "FAILED", "error": res["error"]}
        data = res.get("data", {})
        return {
            "validation_status": "PASSED",
            "customer_id": data.get("customer_id"),
            "loan_amount": data.get("loan_amount"),
            "income": data.get("income"),
            "credit_history": data.get("credit_history")
        }
    except Exception as e:
        return {"error": str(e)}

# Node 2: Check Credit
def node_check_credit(state: LoanGraphState):
    print("--- NODE: Checking Credit ---")
    if state.get("error"): return {}
    try:
        res = json.loads(query_credit_bureau_api_tool._run(state["customer_id"]))
        return {"credit_score": res.get("credit_score", -1)}
    except Exception as e:
        return {"error": str(e)}

# Node 3: Assess Risk (LLM-Powered)
def node_assess_risk(state: LoanGraphState):
    print("--- NODE: Assessing Risk (LLM) ---")
    if state.get("error"): return {}

    prompt = f"Assess risk for: Credit {state['credit_score']}, Amount {state['loan_amount']}. Return LOW, MEDIUM, or HIGH."
    try:
        response = lg_llm.invoke(prompt)
        risk_level = "HIGH" if "HIGH" in response.content else "LOW"
        score = 9 if risk_level == "HIGH" else 3
        return {"risk_level": risk_level, "risk_score": score}
    except Exception as e:
        return {"error": str(e)}

# Node 4: Check Compliance
def node_check_compliance(state: LoanGraphState):
    print("--- NODE: Checking Compliance ---")
    if state.get("error"): return {}
    res = json.loads(check_lending_compliance_tool._run(state["credit_history"], state["risk_score"]))
    return {"compliance_status": res.get("reason", "Unknown")}

# --- IMPROVED REPORTING NODE ---
def node_compile_report(state: LoanGraphState):
    print("--- NODE: Compiling Rich Report ---")
    decision = "Approve" if state.get("risk_level") == "LOW" and not state.get("error") else "Deny"

    # Creating a Rich Markdown Report similar to CrewAI
    report = f"""
# 🏦 Final Loan Decision Report

## 👤 Applicant Information
* **Customer ID:** {state.get('customer_id', 'Unknown')}
* **Loan Amount:** ${state.get('loan_amount', '0')}
* **Income:** {state.get('income', 'Unknown')}

## 📊 Assessment Results
### 1. Credit Check
* **Score:** {state.get('credit_score', 'N/A')}

### 2. Risk Assessment
* **Level:** {state.get('risk_level', 'N/A')}
* **Internal Score:** {state.get('risk_score', 'N/A')}

### 3. Compliance
* **Status:** {state.get('compliance_status', 'Pending')}

---
## ✅ Final Decision: **{decision}**
"""
    return {"final_decision": report}

# --- IMPROVED REJECTION NODE ---
def node_compile_rejection(state: LoanGraphState):
    print("--- NODE: Compiling Rejection Report ---")
    error_msg = state.get('error', 'Unknown Error')
    report = f"""
# 🛑 Loan Application Rejected (Early Termination)

**Reason for Rejection:**
> {error_msg}

**Process Status:**
* **Validation:** {state.get('validation_status', 'Not Started')}
* **Document:** {state.get('document_id', 'Unknown')}
"""
    return {"final_decision": report}

In [18]:
#@title Define & Compile Graph
from langgraph.graph import StateGraph, END

workflow = StateGraph(LoanGraphState)
workflow.add_node("fetch", node_fetch_document)
workflow.add_node("validate", node_validate_document)
workflow.add_node("credit", node_check_credit)
workflow.add_node("risk", node_assess_risk)
workflow.add_node("compliance", node_check_compliance)
workflow.add_node("report", node_compile_report)
workflow.add_node("reject", node_compile_rejection)

workflow.set_entry_point("fetch")

def check_error(state):
    return "reject" if state.get("error") else "continue"

workflow.add_conditional_edges("fetch", check_error, {"continue": "validate", "reject": "reject"})
workflow.add_conditional_edges("validate", check_error, {"continue": "credit", "reject": "reject"})
workflow.add_edge("credit", "risk")
workflow.add_edge("risk", "compliance")
workflow.add_edge("compliance", "report")
workflow.add_edge("report", END)
workflow.add_edge("reject", END)

app = workflow.compile()
print("Graph compiled.")

Graph compiled.


In [19]:
#@title Run LangGraph (Happy Path)

inputs_valid = {
    "applicant_id": "borrower_good_780",
    "document_id": "document_valid_123"
}

try:
    print("--- RUNNING LANGGRAPH ---")

    # WRAPPING THE GRAPH INVOKE
    # We use invoke() here for simplicity in wrapping,
    # but robust_execute works with stream() generators too if configured carefully.
    final_state = robust_execute(app.invoke, input=inputs_valid)

    print("\n--- FINAL DECISION ---")
    print(final_state.get("final_decision"))

except Exception as e:
    handle_execution_error(e)

--- RUNNING LANGGRAPH ---
  >> [Clock 01:40:34] Executing Agent Action (Safe Mode)...
--- NODE: Fetching Document ---
--- HELPER: Simulating fetch for doc_id: document_valid_123 ---
--- NODE: Validating Document ---
--- TOOL: Validating document fields ---
--- NODE: Checking Credit ---
--- TOOL: Calling Credit Bureau for CUST-12345 ---
--- NODE: Assessing Risk (LLM) ---
--- NODE: Checking Compliance ---
--- TOOL: Compliance Check (Risk: 3) ---
--- NODE: Compiling Rich Report ---

--- FINAL DECISION ---

# 🏦 Final Loan Decision Report

## 👤 Applicant Information
* **Customer ID:** CUST-12345
* **Loan Amount:** $50000
* **Income:** USD 120000 a year

## 📊 Assessment Results
### 1. Credit Check
* **Score:** 810

### 2. Risk Assessment
* **Level:** LOW
* **Internal Score:** 3

### 3. Compliance
* **Status:** Compliant.

---
## ✅ Final Decision: **Approve**



In [20]:
#@title Run LangGraph (Unhappy Path: Low Credit Score)
try:
    print("--- RUNNING LANGGRAPH (LOW CREDIT SCENARIO) ---")

    # We use the new mock document which has VALID fields but points to a BAD borrower
    inputs_risky = {
        "applicant_id": "borrower_risky_999",
        "document_id": "document_risky_789"
    }

    # Execute
    final_state = robust_execute(app.invoke, input=inputs_risky)

    # Report
    print(final_state.get("final_decision"))

except Exception as e:
    handle_execution_error(e)

--- RUNNING LANGGRAPH (LOW CREDIT SCENARIO) ---
  >> [Clock 01:40:43] Executing Agent Action (Safe Mode)...
--- NODE: Fetching Document ---
--- HELPER: Simulating fetch for doc_id: document_risky_789 ---
--- NODE: Validating Document ---
--- TOOL: Validating document fields ---
--- NODE: Checking Credit ---
--- TOOL: Calling Credit Bureau for CUST-99999 ---
--- NODE: Assessing Risk (LLM) ---
--- NODE: Checking Compliance ---
--- TOOL: Compliance Check (Risk: 9) ---
--- NODE: Compiling Rich Report ---

# 🏦 Final Loan Decision Report

## 👤 Applicant Information
* **Customer ID:** CUST-99999
* **Loan Amount:** $50000
* **Income:** USD 40000 a year

## 📊 Assessment Results
### 1. Credit Check
* **Score:** 550

### 2. Risk Assessment
* **Level:** HIGH
* **Internal Score:** 9

### 3. Compliance
* **Status:** Risk Score 9 exceeds limit of 7.

---
## ✅ Final Decision: **Deny**

